In [68]:
#Data casos positivos

import pandas as pd
#!pip install patool pandas
import patoolib
import pandas as pd
import os
from tempfile import TemporaryDirectory


rar_path = "C:/Users/Nicolas Silva/Documents/GitHub/Redes2/positivos_covid.rar"
with TemporaryDirectory() as tmpdir:
    patoolib.extract_archive(rar_path, outdir=tmpdir)
    
    csv_filename = [f for f in os.listdir(tmpdir) if f.endswith('.csv')][0]
    csv_path = os.path.join(tmpdir, csv_filename)

    contagiados = pd.read_csv(csv_path, delimiter=';')

print(contagiados)



INFO patool: Extracting C:/Users/Nicolas Silva/Documents/GitHub/Redes2/positivos_covid.rar ...
INFO patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Nicolas Silva\Documents\GitHub\Redes2\positivos_covid.rar"
INFO patool:     with cwd=C:\Users\NICOLA~1\AppData\Local\Temp\tmplcxxmwpf, input=
INFO patool: ... C:/Users/Nicolas Silva/Documents/GitHub/Redes2/positivos_covid.rar extracted to `C:\Users\NICOLA~1\AppData\Local\Temp\tmplcxxmwpf'.


         FECHA_CORTE DEPARTAMENTO  PROVINCIA              DISTRITO METODODX  \
0           20241203       TUMBES     TUMBES                TUMBES       AG   
1           20241203         LIMA       LIMA           JESUS MARIA       AG   
2           20241203   SAN MARTIN  MOYOBAMBA             MOYOBAMBA       AG   
3           20241203     AREQUIPA   CAYLLOMA             COPORAQUE       AG   
4           20241203         LIMA       LIMA           JESUS MARIA       AG   
...              ...          ...        ...                   ...      ...   
4585355     20241203         LIMA       LIMA           JESUS MARIA       AG   
4585356     20241203         LIMA       LIMA           EL AGUSTINO       AG   
4585357     20241203         LIMA       LIMA                   ATE       AG   
4585358     20241203         LIMA       LIMA  SAN MARTIN DE PORRES       AG   
4585359     20241203         LIMA       LIMA            CARABAYLLO       AG   

         EDAD       SEXO  FECHA_RESULTADO    UBIGEO

In [69]:
#data fallecidos
fallecidos = pd.read_csv("fallecidos_covid.csv",delimiter=';')
fallecidos.head()


,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,UUID
0,20240317,20220219,63,MASCULINO,Criterio virológico,TUMBES,TUMBES,TUMBES,240101.0,203506.0
1,20240317,20210529,74,MASCULINO,Criterio virológico,TUMBES,TUMBES,TUMBES,240101.0,203532.0
2,20240317,20210623,72,FEMENINO,Criterio SINADEF,TACNA,TACNA,ALTO DE LA ALIANZA,230102.0,203584.0
3,20240317,20210824,85,MASCULINO,Criterio investigación Epidemiológica,TUMBES,TUMBES,TUMBES,240101.0,212541.0
4,20240317,20210627,46,MASCULINO,Criterio virológico,TACNA,TACNA,ALTO DE LA ALIANZA,230102.0,203665.0


In [70]:
fallecidos["FECHA_FALLECIMIENTO"] = fallecidos["FECHA_FALLECIMIENTO"].astype(str)
fallecidos["year"] = fallecidos["FECHA_FALLECIMIENTO"].str[:4]
fallecidos = fallecidos.drop(columns=["FECHA_FALLECIMIENTO"])

## Contagiados

### Exploramos la data

In [71]:
contagiados.head()


,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20241203,TUMBES,TUMBES,TUMBES,AG,46.0,FEMENINO,20221207.0,240101.0,203499.0
1,20241203,LIMA,LIMA,JESUS MARIA,AG,69.0,FEMENINO,20230822.0,150113.0,221397.0
2,20241203,SAN MARTIN,MOYOBAMBA,MOYOBAMBA,AG,55.0,FEMENINO,20240108.0,220101.0,295651.0
3,20241203,AREQUIPA,CAYLLOMA,COPORAQUE,AG,50.0,MASCULINO,20230824.0,40506.0,851625.0
4,20241203,LIMA,LIMA,JESUS MARIA,AG,58.0,MASCULINO,20221217.0,150113.0,287786.0


In [72]:
contagiados["FECHA_RESULTADO"] = contagiados["FECHA_RESULTADO"].astype(str)
contagiados["year"] = contagiados["FECHA_RESULTADO"].str[:4]
contagiados = contagiados.drop(columns=["FECHA_RESULTADO"])

In [73]:
#Limpiamos la variable year
contagiados["year"].unique()
contagiados = contagiados[~contagiados["year"].isin(['nan', '1899'])]
contagiados = contagiados[contagiados["year"].notna()]

In [74]:
fallecidos['CATEGORIA'] = 'FALLECIDO'
contagiados['CATEGORIA'] = 'CONTAGIADO'
fallecidos = fallecidos.rename(columns={'EDAD_DECLARADA': 'EDAD'})

In [75]:
key_columns = ['year', 'SEXO', 'EDAD', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', 'UBIGEO', 'CATEGORIA']
covid = pd.concat([fallecidos[key_columns], contagiados[key_columns]], ignore_index=True)
print(covid)

         year       SEXO  EDAD DEPARTAMENTO PROVINCIA              DISTRITO  \
0        2022  MASCULINO  63.0       TUMBES    TUMBES                TUMBES   
1        2021  MASCULINO  74.0       TUMBES    TUMBES                TUMBES   
2        2021   FEMENINO  72.0        TACNA     TACNA    ALTO DE LA ALIANZA   
3        2021  MASCULINO  85.0       TUMBES    TUMBES                TUMBES   
4        2021  MASCULINO  46.0        TACNA     TACNA    ALTO DE LA ALIANZA   
...       ...        ...   ...          ...       ...                   ...   
4803856  2022  MASCULINO  21.0         LIMA      LIMA           JESUS MARIA   
4803857  2022  MASCULINO  22.0         LIMA      LIMA           EL AGUSTINO   
4803858  2022  MASCULINO  22.0         LIMA      LIMA                   ATE   
4803859  2022  MASCULINO  22.0         LIMA      LIMA  SAN MARTIN DE PORRES   
4803860  2022  MASCULINO  22.0         LIMA      LIMA            CARABAYLLO   

           UBIGEO   CATEGORIA  
0        240101.0  

In [76]:
covid['CATEGORIA'].value_counts()

CATEGORIA
CONTAGIADO    4582943
FALLECIDO      220918
Name: count, dtype: int64

## Reshaping to Long

People per level, by distrit by year:

In [78]:
indexList = ['year', 'DEPARTAMENTO', 'PROVINCIA',"CATEGORIA"]
aggregator = {'CATEGORIA': [len]}  
covid_provYear = covid.groupby(indexList, observed=True).agg(aggregator)
# Renombramos len a fallecidos
covid_provYear

CATEGORIA
                                                    len
year DEPARTAMENTO PROVINCIA        CATEGORIA           
2020 AMAZONAS     BAGUA            CONTAGIADO      8126
                                   FALLECIDO        270
                  BONGARA          CONTAGIADO       389
                                   FALLECIDO         25
                  CHACHAPOYAS      CONTAGIADO      2174
...                                                 ...
2024 TUMBES       TUMBES           CONTAGIADO        57
                  ZARUMILLA        CONTAGIADO        16
     UCAYALI      CORONEL PORTILLO CONTAGIADO       117
                  EN INVESTIGACIÓN CONTAGIADO         1
                  PADRE ABAD       CONTAGIADO         6

[1866 rows x 1 columns]

Sending the counts to wide columns:

In [79]:
CovidDraft=covid_provYear.unstack(3).fillna(0) #leftmost index in rows
CovidDraft

CATEGORIA          
                                          len          
CATEGORIA                          CONTAGIADO FALLECIDO
year DEPARTAMENTO PROVINCIA                            
2020 AMAZONAS     BAGUA                8126.0     270.0
                  BONGARA               389.0      25.0
                  CHACHAPOYAS          2174.0      61.0
                  CONDORCANQUI         3481.0      79.0
                  EN INVESTIGACIÓN       77.0       0.0
...                                       ...       ...
2024 TUMBES       TUMBES                 57.0       0.0
                  ZARUMILLA              16.0       0.0
     UCAYALI      CORONEL PORTILLO      117.0       0.0
                  EN INVESTIGACIÓN        1.0       0.0
                  PADRE ABAD              6.0       0.0

[1078 rows x 2 columns]

Computing share of covid, level 'Fallecido':

In [80]:
CovidDraft['Fallecido_pct']=CovidDraft.iloc[:,1]/(CovidDraft.iloc[:,0] + CovidDraft.iloc[:,1])
Covid_provYear_Fallecido_w=CovidDraft['Fallecido_pct'].unstack('year').fillna(0)
Covid_provYear_Fallecido_w

year                               2020      2021      2022      2023  \
DEPARTAMENTO PROVINCIA                                                  
AMAZONAS     BAGUA             0.032158  0.043343  0.009309  0.012048   
             BONGARA           0.060386  0.042365  0.004338  0.000000   
             CHACHAPOYAS       0.027293  0.039357  0.003565  0.019868   
             CONDORCANQUI      0.022191  0.046713  0.009671  0.000000   
             EN INVESTIGACIÓN  0.000000  0.000000  0.000000  0.000000   
...                                 ...       ...       ...       ...   
UCAYALI      ATALAYA           0.083491  0.230216  0.040541  0.000000   
             CORONEL PORTILLO  0.083943  0.119762  0.010725  0.081545   
             EN INVESTIGACIÓN  0.000000  0.000000  0.000000  0.000000   
             PADRE ABAD        0.050963  0.070412  0.009662  0.000000   
             PURUS             0.005666  0.045455  0.000000  0.000000   

year                               2024  
DEPARTAMENTO PROVINCIA                   
AMAZONAS     BAGUA             0.000000  
             BONGARA           0.000000  
             CHACHAPOYAS       0.007299  
             CONDORCANQUI      0.000000  
             EN INVESTIGACIÓN  0.000000  
...                                 ...  
UCAYALI      ATALAYA           0.000000  
             CORONEL PORTILLO  0.000000  
             EN INVESTIGACIÓN  0.000000  
             PADRE ABAD        0.000000  
             PURUS             0.000000  

[225 rows x 5 columns]

Notice the data type:

In [81]:
Covid_provYear_Fallecido_w.columns

Index(['2020', '2021', '2022', '2023', '2024'], dtype='object', name='year')

In [82]:
Covid_provYear_Fallecido_w.columns=['year'+str(x) for x in Covid_provYear_Fallecido_w.columns]

In [83]:
# then
Covid_provYear_Fallecido_w

year2020  year2021  year2022  year2023  \
DEPARTAMENTO PROVINCIA                                                  
AMAZONAS     BAGUA             0.032158  0.043343  0.009309  0.012048   
             BONGARA           0.060386  0.042365  0.004338  0.000000   
             CHACHAPOYAS       0.027293  0.039357  0.003565  0.019868   
             CONDORCANQUI      0.022191  0.046713  0.009671  0.000000   
             EN INVESTIGACIÓN  0.000000  0.000000  0.000000  0.000000   
...                                 ...       ...       ...       ...   
UCAYALI      ATALAYA           0.083491  0.230216  0.040541  0.000000   
             CORONEL PORTILLO  0.083943  0.119762  0.010725  0.081545   
             EN INVESTIGACIÓN  0.000000  0.000000  0.000000  0.000000   
             PADRE ABAD        0.050963  0.070412  0.009662  0.000000   
             PURUS             0.005666  0.045455  0.000000  0.000000   

                               year2024  
DEPARTAMENTO PROVINCIA                   
AMAZONAS     BAGUA             0.000000  
             BONGARA           0.000000  
             CHACHAPOYAS       0.007299  
             CONDORCANQUI      0.000000  
             EN INVESTIGACIÓN  0.000000  
...                                 ...  
UCAYALI      ATALAYA           0.000000  
             CORONEL PORTILLO  0.000000  
             EN INVESTIGACIÓN  0.000000  
             PADRE ABAD        0.000000  
             PURUS             0.000000  

[225 rows x 5 columns]

In [84]:
# as usual
Covid_provYear_Fallecido_w.reset_index(inplace=True)
Covid_provYear_Fallecido_w

,DEPARTAMENTO,PROVINCIA,year2020,year2021,year2022,year2023,year2024
0,AMAZONAS,BAGUA,0.032158,0.043343,0.009309,0.012048,0.000000
1,AMAZONAS,BONGARA,0.060386,0.042365,0.004338,0.000000,0.000000
2,AMAZONAS,CHACHAPOYAS,0.027293,0.039357,0.003565,0.019868,0.007299
3,AMAZONAS,CONDORCANQUI,0.022191,0.046713,0.009671,0.000000,0.000000
4,AMAZONAS,EN INVESTIGACIÓN,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
220,UCAYALI,ATALAYA,0.083491,0.230216,0.040541,0.000000,0.000000
221,UCAYALI,CORONEL PORTILLO,0.083943,0.119762,0.010725,0.081545,0.000000
222,UCAYALI,EN INVESTIGACIÓN,0.000000,0.000000,0.000000,0.000000,0.000000
223,UCAYALI,PADRE ABAD,0.050963,0.070412,0.009662,0.000000,0.000000


Let's call a map:

In [ ]:
mapLink='https://github.com/SocialAnalytics-StrategicIntelligence/GeoDF_Analytics/raw/main/maps/ProvsINEI2023.zip'

import geopandas as gpd

provmap=gpd.read_file(mapLink)

provmap.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    196 non-null    float64 
 1   CCDD        196 non-null    object  
 2   CCPP        196 non-null    object  
 3   DEPARTAMEN  196 non-null    object  
 4   PROVINCIA   196 non-null    object  
 5   geometry    196 non-null    geometry
dtypes: float64(1), geometry(1), object(4)
memory usage: 9.3+ KB


Let me create a column, concatenating two:

In [ ]:
provmap['location']=['+'.join(x[0]) for x in zip(provmap.iloc[:,3:5].values)]
provmap.head(10)

,OBJECTID,CCDD,CCPP,DEPARTAMEN,PROVINCIA,geometry,location
0,1.0,01,01,AMAZONAS,CHACHAPOYAS,"POLYGON ((-77.72614 -5.94354, -77.72486 -5.943...",AMAZONAS+CHACHAPOYAS
1,2.0,01,02,AMAZONAS,BAGUA,"POLYGON ((-78.61909 -4.51001, -78.61802 -4.510...",AMAZONAS+BAGUA
2,3.0,01,03,AMAZONAS,BONGARA,"POLYGON ((-77.72759 -5.14030, -77.72361 -5.140...",AMAZONAS+BONGARA
3,4.0,01,04,AMAZONAS,CONDORCANQUI,"POLYGON ((-77.81399 -2.99278, -77.81483 -2.995...",AMAZONAS+CONDORCANQUI
4,5.0,01,05,AMAZONAS,LUYA,"POLYGON ((-78.13023 -5.90370, -78.13011 -5.904...",AMAZONAS+LUYA
5,6.0,01,06,AMAZONAS,RODRIGUEZ DE MENDOZA,"POLYGON ((-77.44452 -6.05002, -77.44387 -6.050...",AMAZONAS+RODRIGUEZ DE MENDOZA
6,7.0,01,07,AMAZONAS,UTCUBAMBA,"POLYGON ((-78.09288 -5.36258, -78.09288 -5.364...",AMAZONAS+UTCUBAMBA
7,8.0,02,01,ANCASH,HUARAZ,"POLYGON ((-77.39870 -9.35563, -77.39852 -9.356...",ANCASH+HUARAZ
8,9.0,02,02,ANCASH,AIJA,"POLYGON ((-77.61368 -9.64900, -77.61241 -9.649...",ANCASH+AIJA
9,10.0,02,03,ANCASH,ANTONIO RAYMONDI,"POLYGON ((-77.08856 -8.97496, -77.08804 -8.975...",ANCASH+ANTONIO RAYMONDI


I will do the same with the data frame:

In [ ]:
dengue_provYear_Alarm_w['location']=['+'.join(x[0]) for x in zip(dengue_provYear_Alarm_w.iloc[:,:2].values)]
dengue_provYear_Alarm_w.head()

,departamento,provincia,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021,year2022,location
0,AMAZONAS,BAGUA,0.154519,0.015385,0.054545,0.000000,0.000000,0.000000,0.011111,0.040323,0.083333,0.075721,0.077234,AMAZONAS+BAGUA
1,AMAZONAS,BONGARA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,AMAZONAS+BONGARA
2,AMAZONAS,CHACHAPOYAS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.500000,0.125000,AMAZONAS+CHACHAPOYAS
3,AMAZONAS,CONDORCANQUI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153285,0.028070,0.012500,AMAZONAS+CONDORCANQUI
4,AMAZONAS,UTCUBAMBA,0.047368,0.034091,0.086667,0.086957,0.121212,0.055556,0.000000,0.000000,0.065268,0.028290,0.047821,AMAZONAS+UTCUBAMBA


## Preprocessing



The names from non-english speaking countries may come with some symbols that may cause trouble (', ~). Let's get rid of those:

In [ ]:
import unidecode


byePunctuation=lambda x: unidecode.unidecode(x)
dengue_provYear_Alarm_w['location']=dengue_provYear_Alarm_w['location'].apply(byePunctuation)
provmap['location']=provmap['location'].apply(byePunctuation)

ModuleNotFoundError: No module named 'unidecode'

It would be good making sure no *ghost* appears between words:

In [ ]:
# replacing dashes and multiple spaces by a simple space
dengue_provYear_Alarm_w['location']=dengue_provYear_Alarm_w.location.str.replace("\-|\_|\s+","",regex=True)
provmap['location']=provmap.location.str.replace("\-|\_|\s+","",regex=True)

## Merging

We need to merge both tables now. That can happen effectively if both tables have a **key** column: a column (or collection of them) whose values in one table are the same in the other one.

The match need not be exact, but only common values in the *key* are merged.

Let's find out what is NOT matched in each table:

In [ ]:
nomatch_df=set(dengue_provYear_Alarm_w.location)- set(provmap.location)
nomatch_gdf=set(provmap.location)-set(dengue_provYear_Alarm_w.location)

This is what could not be matched:

In [ ]:
len(nomatch_df), len(nomatch_gdf)

The right way to go is using **fuzzy merging** (remember we need  _the fuzz_):

In [ ]:
# pick the closest match from nomatch_gdf for a value in nomatch_df
from thefuzz import process
[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

If you are comfortable, you prepare a _dictionary_ of changes:

In [ ]:
# is this OK?
{dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

In [ ]:
# then:
changesinDF={dis:process.extractOne(dis,nomatch_gdf)[0] for dis in sorted(nomatch_df)}

Now, make the replacements:

In [ ]:
dengue_provYear_Alarm_w.replace({'location': changesinDF}, inplace=True)

Is it over?

In [ ]:
nomatch_df=set(dengue_provYear_Alarm_w.location)- set(provmap.location)
nomatch_gdf=set(provmap.location)-set(dengue_provYear_Alarm_w.location)

[(dis,process.extractOne(dis,nomatch_gdf)) for dis in sorted(nomatch_df)]

Now the merge can happen:

In [ ]:
dengue_provYear_Alarm_map=provmap.merge(dengue_provYear_Alarm_w, on='location',how='left',indicator='flag')

In [ ]:
# check
dengue_provYear_Alarm_map.info()

In [ ]:
# avoid poblems with fillna()
dengue_provYear_Alarm_map['flag']=dengue_provYear_Alarm_map.flag.astype(str)

We can get rid of some columns:

In [ ]:
bye=['departamento', 'provincia', 'CCPP','CCDD']
dengue_provYear_Alarm_map.drop(columns=bye,inplace=True)

# keeping
dengue_provYear_Alarm_map.head()

In [ ]:
# filling with zeroes
dengue_provYear_Alarm_map.fillna(0,inplace=True)

We can save this geoDF:

In [ ]:
import os
dengue_provYear_Alarm_map.to_file(os.path.join('maps',"provinciasPeru.gpkg"), layer='provinciasDengue', driver="GPKG")

## Exploring one variable

This time, we explore statistically one variable in the map:

In [ ]:
# statistics
dengue_provYear_Alarm_map.year2022.describe()

A visual look:

In [ ]:
import seaborn as sea

sea.boxplot(dengue_provYear_Alarm_map.year2022, color='yellow',orient='h')

In [ ]:

from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=100, random_state=0,output_distribution='normal')
qt_result=qt.fit_transform(dengue_provYear_Alarm_map[['year2022']])
sea.boxplot(qt_result, color='yellow',orient='h')

In [ ]:
dengue_provYear_Alarm_map['year_2022_qt']=qt_result

## Spatial Correlation

### Neighboorhood

We can compute the neighborhood in a map using different algorithms:

In [ ]:
from libpysal.weights import Queen, Rook, KNN

# rook

w_rook = Rook.from_dataframe(dengue_provYear_Alarm_map,use_index=False)

In [ ]:
# rook
w_queen = Queen.from_dataframe(dengue_provYear_Alarm_map,use_index=False)

In [ ]:
# k nearest neighbors
w_knn = KNN.from_dataframe(dengue_provYear_Alarm_map, k=8)

Let's understand the differences:

In [ ]:
# first one
dengue_provYear_Alarm_map.head(1)

In [ ]:
# amount neighbors of that district
w_rook.neighbors[0]

In [ ]:
# see
base=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.PROVINCIA=="CHACHAPOYAS"].plot()
dengue_provYear_Alarm_map.iloc[w_rook.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_provYear_Alarm_map.head(1).plot(ax=base,facecolor="red")

Let's do the same:

In [ ]:
w_queen.neighbors[0]

In [ ]:
base=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.PROVINCIA=="CHACHAPOYAS"].plot()
dengue_provYear_Alarm_map.iloc[w_queen.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_provYear_Alarm_map.head(1).plot(ax=base,facecolor="red")

In [ ]:
w_knn.neighbors[0]

In [ ]:
base=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.PROVINCIA=="CHACHAPOYAS"].plot()
dengue_provYear_Alarm_map.iloc[w_knn.neighbors[0] ,].plot(ax=base,facecolor="yellow",edgecolor='k')
dengue_provYear_Alarm_map.head(1).plot(ax=base,facecolor="red")

Let me pay attention to the queen results:

In [ ]:
# all the neighbors by row
w_queen.neighbors

In [ ]:
# the matrix of neighboorhood:

pd.DataFrame(*w_queen.full()).astype(int) # 1 means both are neighbors

In [ ]:
# pct of neighboorhood (density)
w_queen.pct_nonzero

In [ ]:
# a province with NO neighbor?
w_queen.islands

## Moran's correlation

We need the neighboorhood matrix (the weight matrix) to compute spatial correlation: if the variable value is correlated with the values of its neighbors - which proves a spatial effect.

In [ ]:
# needed for spatial correlation
w_queen.transform = 'R'

In [ ]:
pd.DataFrame(*w_queen.full()).sum(axis=1) # 1 means both are neighbors

Spatial correlation is measured by the Moran's I statistic:

In [ ]:
from esda.moran import Moran

moranDENGUE = Moran(dengue_provYear_Alarm_map['year_2022_qt'], w_queen)
moranDENGUE.I,moranDENGUE.p_sim

The Moran's I is significant. Let's see:

In [ ]:
from splot.esda import moran_scatterplot
import matplotlib.pyplot as plt

fig, ax = moran_scatterplot(moranDENGUE)
ax.set_xlabel('Dengue_alarma_share')
ax.set_ylabel('SpatialLag_Dengue_alarma_share')


### Local Spatial Correlation

We can compute a LISA (local Moran) for each case. That will help us find spatial clusters (spots) and spatial outliers:

* A **hotSpot** is a polygon whose value in the variable is high AND is surrounded with polygons with also high values.

* A **coldSpot** is a polygon whose value in the variable is low AND is surrounded with polygons with also low values.

* A **coldOutlier** is a polygon whose value in the variable is low BUT is surrounded with polygons with  high values.

* A **hotOutlier** is a polygon whose value in the variable is high BUT is surrounded with polygons with  low values.

It is also possible that no significant correlation is detected. Let's see those values:

In [ ]:
# The scatterplot with local info

from esda.moran import Moran_Local

# calculate Moran_Local and plot
lisaDENGUE = Moran_Local(y=dengue_provYear_Alarm_map['year_2022_qt'], w=w_knn,seed=2022)
fig, ax = moran_scatterplot(lisaDENGUE,p=0.05)
ax.set_xlabel('Dengue_alarma_share')
ax.set_ylabel('SpatialLag_Dengue_alarma_share');


In [ ]:
from splot.esda import plot_local_autocorrelation
plot_local_autocorrelation(lisaDENGUE, dengue_provYear_Alarm_map,'year_2022_qt')
plt.show()

In [ ]:
# the map with the spots and outliers

from splot.esda import lisa_cluster
f, ax = plt.subplots(1, figsize=(12, 12))
plt.title('Spots and Outliers')
fig = lisa_cluster(lisaDENGUE,
                   dengue_provYear_Alarm_map,ax=ax,
                   legend_kwds={'loc': 'center left',
                                'bbox_to_anchor': (0.7, 0.6)})

Let me add that data to my gdf:

In [ ]:
# quadrant
lisaDENGUE.q

In [ ]:
# significance
lisaDENGUE.p_sim

In [ ]:
# quadrant: 1 HH,  2 LH,  3 LL,  4 HL
pd.Series(lisaDENGUE.q).value_counts()

The info in **lisaDENGUE.q** can not be used right away, we need to add if the local spatial correlation is significant:

In [ ]:
dengue_provYear_Alarm_map['DENGUE_quadrant']=[l if p <0.05 else 0 for l,p in zip(lisaDENGUE.q,lisaDENGUE.p_sim)  ]
dengue_provYear_Alarm_map['DENGUE_quadrant'].value_counts()

Now, we recode:

In [ ]:
labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

dengue_provYear_Alarm_map['DENGUE_quadrant_names']=[labels[i] for i in dengue_provYear_Alarm_map['DENGUE_quadrant']]

dengue_provYear_Alarm_map['DENGUE_quadrant_names'].value_counts()


Let's replot:

In [ ]:
from matplotlib import colors
myColMap = colors.ListedColormap([ 'ghostwhite', 'red', 'green', 'black','orange'])




f, ax = plt.subplots(1, figsize=(12,12))


plt.title('Spots and Outliers')

dengue_provYear_Alarm_map.plot(column='DENGUE_quadrant_names',
                categorical=True,
                cmap=myColMap,
                linewidth=0.1,
                edgecolor='white',
                legend=True,
                legend_kwds={'loc': 'center left',
                             'bbox_to_anchor': (0.7, 0.6)},
                ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
dengue_provYear_Alarm_map.explore("DENGUE_quadrant_names", categorical=True,tooltip='location',cmap=myColMap)

In [ ]:
import folium

map1=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.DENGUE_quadrant_names=='1 hotSpot']
map2=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.DENGUE_quadrant_names=='2 coldOutlier']
map3=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.DENGUE_quadrant_names=='3 coldSpot']
map4=dengue_provYear_Alarm_map[dengue_provYear_Alarm_map.DENGUE_quadrant_names=='4 hotOutlier']

m = map1.explore(
    color="red",
    tooltip=False,  # hide tooltip
    popup=["location"],  # (on-click)
    name="hotSpot"  # name of the layer in the map
)

map2.explore(
    m=m, # notice
    color="green",
    tooltip=False,
    popup=["location"],
    name="coldOutlier"
)

map3.explore(
    m=m,
    color="black",
    tooltip=False,
    popup=["location"],
    name="coldSpot",
)

map4.explore(
    m=m,
    color="orange",
    tooltip=False,
    popup=["location"],
    name="hotOutlier",
)

folium.TileLayer("CartoDB positron", show=False).add_to(m)  # use folium to add alternative tiles
folium.LayerControl(collapsed=True).add_to(m)  # use folium to add layer control

m  # show map

## Bivariate LISA

In [ ]:
#from esda.moran import Moran_BV, Moran_Local_BV
from esda.moran import Moran_BV

mbi = Moran_BV(dengue_provYear_Alarm_map['year2021'],  dengue_provYear_Alarm_map['year2022'],  w_queen)
mbi.I,mbi.p_sim

In [ ]:
# The scatterplot with local info
from esda.moran import Moran_Local_BV

# calculate Moran_Local and plot
lisaDENGUE_bv = Moran_Local_BV(y=dengue_provYear_Alarm_map['year2021'],
                               x=dengue_provYear_Alarm_map['year2022'],
                               w=w_queen)

fig, ax = moran_scatterplot(lisaDENGUE_bv, p=0.05,aspect_equal=True)

ax.set_xlabel('Dengue_2022')
ax.set_ylabel('SpatialLag_Dengue_2021')
plt.show()

In [ ]:
dengue_provYear_Alarm_map['DENGUE_quadrant_21_22']=[l if p <0.05 else 0 for l,p in zip(lisaDENGUE_bv.q,lisaDENGUE_bv.p_sim)  ]

labels = [ '0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier']

dengue_provYear_Alarm_map['DENGUE_quadrant_21_22_names']=[labels[i] for i in dengue_provYear_Alarm_map['DENGUE_quadrant_21_22']]


In [ ]:
# see new columns
dengue_provYear_Alarm_map

In [ ]:
from matplotlib import colors
myColMap = colors.ListedColormap([ 'ghostwhite', 'red', 'green', 'black','orange'])




f, ax = plt.subplots(1, figsize=(12,12))


plt.title('Spots and Outliers')

dengue_provYear_Alarm_map.plot(column='DENGUE_quadrant_21_22_names',
                categorical=True,
                cmap=myColMap,
                linewidth=0.1,
                edgecolor='white',
                legend=True,
                legend_kwds={'loc': 'center left',
                             'bbox_to_anchor': (0.7, 0.6)},
                ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()